In [13]:
import torch as t
from torch.utils.data import Dataset , DataLoader
from torch.nn.utils.rnn import pad_sequence 
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize , word_tokenize

In [14]:
df = pd.read_csv("spam.csv" , encoding="latin1")
to_drop = ["Unnamed: 2","Unnamed: 3","Unnamed: 4"]
df.drop(columns=to_drop , inplace=True)

# lets encode v1
df["v1"] = [0 if item == "ham" else 1 for item in df["v1"]]
df

sents = []
for sentences in df["v2"] :
    words = word_tokenize(sentences)
    sents.append(words)

stop_words = stopwords.words("english")
# lets FLATTEN THIS BIG BOY AND REMOVE STOP_WRODS
all_words = [word for arr in sents for word in arr if word not in stop_words]
# MAKE EVERY WORD UNIQUE LIKE U BABY
all_words = set(all_words)
# IDS for every word
vocab = {word:idx for idx,word in enumerate(all_words)}
# apply ids for every word in every sentences

z = []
for sentence in df["v2"]:
    sen = []
    words = word_tokenize(sentence)
    for word in words :
        if word in vocab.keys():
            sen.append(vocab[word])
    
    z.append(sen)
    

df["v3"] = z

df.drop(columns=["v2"] , inplace=True)

df



          
seq_pad = [t.tensor(idx) for idx in df["v3"]]

seqs_padded = pad_sequence(seq_pad , True , 0)
print(seqs_padded.shape)# (vocab_size , ndim)
len(vocab)

torch.Size([5572, 211])


11389

In [15]:
class EmbeddingModel(t.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = t.nn.Embedding(vocab_size, embed_dim)

  
        self.network = t.nn.Sequential(
            t.nn.Linear(embed_dim, 128),  
            t.nn.ReLU(),
            t.nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x)        # (batch_size, seq_len, embed_dim)
        x = x.mean(dim=1)            # (batch_size, embed_dim)
        x = self.network(x)          # (batch_size, num_classes)
        return x



from torch.utils.data import TensorDataset , DataLoader
from sklearn.model_selection import train_test_split

labels = df["v1"].to_numpy() 

X_train , X_test , Y_train , Y_test = train_test_split(seqs_padded , labels , random_state=42)

import torch

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long),
                              torch.tensor(Y_train, dtype=torch.long))

test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.long),
                             torch.tensor(Y_test, dtype=torch.long))

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18916\4154259663.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long),
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18916\4154259663.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.long),


In [16]:

model = EmbeddingModel(11389, 211, 2)  # 207 = embed_dim
epochs = 15
criterion = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    for batch in train_loader:
        batch_x, batch_y = batch  # فرقنا الـ input عن الـ labels
        
        optimizer.zero_grad()      # مهم جداً قبل كل backward
        y_pred = model(batch_x)    # forward pass
        
        loss = criterion(y_pred, batch_y)  # حساب الخسارة
        loss.backward()            # backward pass
        optimizer.step()           # تحديث الأوزان

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
    
    

Epoch 1, Loss: 0.6547
Epoch 2, Loss: 0.3463
Epoch 3, Loss: 0.3075
Epoch 4, Loss: 0.6042
Epoch 5, Loss: 0.1266
Epoch 6, Loss: 0.1241
Epoch 7, Loss: 0.0191
Epoch 8, Loss: 0.0491
Epoch 9, Loss: 0.0063
Epoch 10, Loss: 0.0204
Epoch 11, Loss: 0.0788
Epoch 12, Loss: 0.0880
Epoch 13, Loss: 0.0189
Epoch 14, Loss: 0.0029
Epoch 15, Loss: 0.0056


In [19]:
model.eval()  


sentence = "please write your credit card information in that website https:\\www.site.com"

# tokenize + lowercase + convert to IDs
tokens = word_tokenize(sentence.lower())
ids = [vocab[w] for w in tokens if w in vocab] 

x_input = torch.tensor(ids).unsqueeze(0)  


seq_len = 50 
if x_input.shape[1] < seq_len:
    pad_len = seq_len - x_input.shape[1]
    x_input = torch.cat([x_input, torch.zeros(1, pad_len, dtype=torch.long)], dim=1)
elif x_input.shape[1] > seq_len:
    x_input = x_input[:, :seq_len]  


with torch.no_grad():
    y_pred = model(x_input) 
    predicted_class = torch.argmax(y_pred, dim=1).item()

classes = {0: "ham", 1: "spam"}
print(f"Predicted class: {classes[predicted_class]}")


with torch.no_grad():
    sentence_embedding = model.embedding(x_input).mean(dim=1) 
print("Sentence embedding shape:", sentence_embedding.shape)


Predicted class: spam
Sentence embedding shape: torch.Size([1, 211])
